In [1]:
import fibre
import numpy as np

from __future__ import print_function
logger = fibre.utils.Logger(verbose=True)

## 机械臂的SN码可以通过`lsusb`命令查看

In [ ]:
teach_arm = fibre.find_any(serial_number="208C31875253", logger=logger)

In [ ]:
teach_arm.robot.hand.angle

## RPC连上后,每一个层级都可以直接打印查看

In [ ]:
teach_arm

In [ ]:
teach_arm.robot

In [ ]:
teach_arm.robot.joint_1

In [ ]:
teach_arm.robot.hand

## 接着让我们来测试一些功能

### 机械臂MoveJ

In [ ]:
teach_arm.robot.set_enable(True)
teach_arm.robot.move_j(0, 0, 90, 0, 0, 0)

### 获取所有关节角度(获取到的这个角度是电机的关节机械角, 和运动学模型角度之间存在一个固定的offset)

In [38]:
joint_offset = np.array([0,-73,180,0,0,0])

In [ ]:
all_joint_angles = teach_arm.robot.joint_1.angle, teach_arm.robot.joint_2.angle, teach_arm.robot.joint_3.angle, teach_arm.robot.joint_4.angle, teach_arm.robot.joint_5.angle, teach_arm.robot.joint_6.angle
all_joint_angles = np.round(np.array(all_joint_angles))
all_joint_angles = all_joint_angles + joint_offset
all_joint_angles

### 获取当前末端位姿(旋转使用ZYX欧拉角序列(也称为RPY - Roll, Pitch, Yaw))

In [ ]:
def get_current_pose():
    teach_arm.robot.eef_pose.update_pose_6D()
    x,y,z,rz,ry,rx = teach_arm.robot.eef_pose.x, teach_arm.robot.eef_pose.y, teach_arm.robot.eef_pose.z, teach_arm.robot.eef_pose.a, teach_arm.robot.eef_pose.b, teach_arm.robot.eef_pose.c
    current_pose = np.array([x,y,z,rz,ry,rx])
    return current_pose

current_pose = get_current_pose()
current_pose


### 机械臂MoveL(直接控制末端在笛卡尔空间运动, 输入单位是mm)

In [ ]:
# 沿着x轴方向移动10mm
teach_arm.robot.move_l(current_pose[0]+10, current_pose[1], current_pose[2], current_pose[3], current_pose[4], current_pose[5])
current_pose = get_current_pose()
current_pose

In [ ]:
# 沿着z轴方向移动10mm
teach_arm.robot.move_l(current_pose[0], current_pose[1], current_pose[2]-10, current_pose[3], current_pose[4], current_pose[5])
current_pose = get_current_pose()
current_pose


### 归巢

In [5]:
teach_arm.robot.resting()

### 复位回到home pose

In [ ]:
teach_arm.robot.move_j(0, 0, 90, 0, 0, 0)

### 示教臂的夹爪实时位置读取

In [6]:
## 让夹爪电机进入零力矩状态
teach_arm.robot.hand.set_mode(0)
teach_arm.robot.hand.set_torque(0)

In [ ]:
teach_arm.robot.set_enable(False)
## 拖动夹爪然后查看角度变化
rate = 2  # 50Hz
def get_current_pose():
    teach_arm.robot.eef_pose.update_pose_6D()
    x,y,z,rz,ry,rx = teach_arm.robot.eef_pose.x, teach_arm.robot.eef_pose.y, teach_arm.robot.eef_pose.z, teach_arm.robot.eef_pose.a, teach_arm.robot.eef_pose.b, teach_arm.robot.eef_pose.c
    current_pose = np.array([x,y,z,rz,ry,rx])
    return current_pose


import time
while True:
    start_time = time.time()
    current_pose = get_current_pose()
    print(current_pose)
    time.sleep(1/rate)
    print(teach_arm.robot.hand.angle)

In [ ]:
teach_arm.robot.set_enable(False)
import time
rate = 0.1  # 10Hz
## 让夹爪电机进入零力矩状态
teach_arm.robot.hand.set_mode(0)
teach_arm.robot.hand.set_torque(0)
while True:
    start_time = time.time()
    joint1_angle = teach_arm.robot.joint_1.angle
    joint2_angle = teach_arm.robot.joint_2.angle
    joint3_angle = teach_arm.robot.joint_3.angle
    joint4_angle = teach_arm.robot.joint_4.angle
    joint5_angle = teach_arm.robot.joint_5.angle
    joint6_angle = teach_arm.robot.joint_6.angle
    # print(f"Current leader Joints: {joint1_angle}, {joint2_angle}, {joint3_angle}, {joint4_angle}, {joint5_angle}, {joint6_angle}")
    all_joint_angles = np.array([joint1_angle, joint2_angle, joint3_angle, joint4_angle, joint5_angle, joint6_angle]) + joint_offset
    all_joint_angles = np.round(all_joint_angles, 0)
    # print("Current gripper: ",teach_arm.robot.hand.angle)
    # joint1_qpos = teach_arm.robot.joint_1.position
    joint1_qvel = teach_arm.robot.joint_1.velocity
    print( "qvel: ", joint1_qvel)
    joint2_qvel = teach_arm.robot.joint_2.velocity
    joint3_qvel = teach_arm.robot.joint_3.velocity
    joint4_qvel = teach_arm.robot.joint_4.velocity
    joint5_qvel = teach_arm.robot.joint_5.velocity
    joint6_qvel = teach_arm.robot.joint_6.velocity
    print( "qvel: ", joint2_qvel, joint3_qvel, joint4_qvel, joint5_qvel, joint6_qvel)
    # follow_arm.robot.move_j(all_joint_angles[0], all_joint_angles[1], all_joint_angles[2], all_joint_angles[3], all_joint_angles[4], all_joint_angles[5])
    # follow_arm.robot.hand.set_angle(teach_arm.robot.hand.angle - teach_hand_init_angle + follow_hand_init_angle)
    # print(f"Current follower Joints (state): {all_joint_angles}")
    # collector.collect_step(
    # teach_joints=np.array([joint1_angle, joint2_angle, joint3_angle, 
    #                         joint4_angle, joint5_angle, joint6_angle]),
    # follow_joints=all_joint_angles,
    # teach_gripper=teach_arm.robot.hand.angle,
    # follow_gripper=follow_arm.robot.hand.angle
    # )
    # Sleep precisely to maintain 10Hz
    elapsed = time.time() - start_time
    if elapsed < rate:
        time.sleep(rate - elapsed)
    
    # Calculate and print actual loop frequency
    loop_time = time.time() - start_time
    actual_freq = 1.0 / loop_time if loop_time > 0 else 0
    print(f"Current loop frequency: {actual_freq:.2f} Hz")

### 示教臂的夹爪实时力矩读取(即将支持,硬件施工中)

### Follow Arm相机测试

In [ ]:
import cv2
mjpg_streamer_url = "http://192.168.65.110:8080/?action=stream"
cap = cv2.VideoCapture(mjpg_streamer_url)
while True:
    ret, frame = cap.read()
    cv2.imshow("frame", frame)
    cv2.waitKey(1)
cv2.destroyAllWindows()

# 校准机械臂初始状态

In [ ]:
from __future__ import print_function
import fibre
logger = fibre.utils.Logger(verbose=True)
SN = "396636713233"
teach_arm = fibre.find_any(serial_number=SN, logger=logger)

In [30]:
teach_arm.robot.resting()

In [ ]:
import numpy as np
def get_current_pose():
    teach_arm.robot.eef_pose.update_pose_6D()
    x,y,z,rz,ry,rx = teach_arm.robot.eef_pose.x, teach_arm.robot.eef_pose.y, teach_arm.robot.eef_pose.z, teach_arm.robot.eef_pose.a, teach_arm.robot.eef_pose.b, teach_arm.robot.eef_pose.c
    current_pose = np.array([x,y,z,rz,ry,rx])
    return current_pose

current_pose = get_current_pose()
current_pose


In [ ]:
teach_arm.robot.set_enable(True)
teach_arm.robot.move_l(current_pose[0], 0.1, current_pose[2], current_pose[3], current_pose[4], current_pose[5])
current_pose = get_current_pose()
current_pose

In [43]:
# Enable motor but disable circle udpate
teach_arm.robot.joint_all.set_enable(True)
teach_arm.robot.set_enable(False)

## 先手动把机械臂挪成L姿态, 然后运行以下代码

In [44]:
teach_arm.robot.joint_1.apply_home_offset()
teach_arm.robot.joint_2.apply_home_offset()
teach_arm.robot.joint_3.apply_home_offset()
teach_arm.robot.joint_4.apply_home_offset()
teach_arm.robot.joint_5.apply_home_offset()
teach_arm.robot.joint_6.apply_home_offset()

## 再手动把机械臂挪回休息状态, 然后运行以下代码

In [45]:
teach_arm.robot.joint_1.apply_home_offset()
teach_arm.robot.joint_2.apply_home_offset()
teach_arm.robot.joint_3.apply_home_offset()
teach_arm.robot.joint_4.apply_home_offset()
teach_arm.robot.joint_5.apply_home_offset()
teach_arm.robot.joint_6.apply_home_offset()

In [ ]:
teach_arm.robot.set_enable(True)
teach_arm.robot.move_j(0, 0, 90, 0, 55, 0)

In [ ]:
teach_arm.robot.set_enable(True)
teach_arm.robot.move_j(-17.21704864501953, 41.4782829284668, 23.368927001953125, 0.0007031249697320163, 70.50114440917969, 2.599015235900879)

In [ ]:
import fibre
import numpy as np

from __future__ import print_function
logger = fibre.utils.Logger(verbose=True)
teach_arm = fibre.find_any(serial_number="208C31875253", logger=logger)
follow_arm = fibre.find_any(serial_number="396636713233", logger=logger)

In [68]:
teach_arm.robot.set_enable(True)
teach_arm.robot.hand.set_mode(0)

In [67]:
teach_arm.robot.hand.set_angle(27)

In [ ]:
print(teach_arm.robot.hand.angle - follow_arm.robot.hand.angle)

In [46]:
teach_arm.robot.hand.set_current_limit(-0.1)

In [25]:
teach_arm.robot.hand.set_mode(0)
teach_arm.robot.hand.set_torque(0)

In [ ]:
teach_arm.robot.hand.get_current_limit()

In [ ]:
follow_arm.robot.hand.get_current_limit()

In [ ]:
teach_arm.robot.hand.set_mode(2)
teach_arm.robot.set_enable(True)
teach_arm.robot.hand.set_angle(27)

In [ ]:
teach_arm.robot.hand.angle

In [ ]:
follow_arm = fibre.find_any(serial_number="396636713233", logger=logger)

In [ ]:
print(follow_arm.robot)

In [7]:
follow_arm.robot.homing()

In [ ]:
follow_arm.robot.move_j(10,10,92,5,10,10)

In [11]:
follow_arm.robot.set_enable(True)

In [ ]:
from __future__ import print_function
import fibre
logger = fibre.utils.Logger(verbose=True)
SN = "396636713233"
teach_arm = fibre.find_any(serial_number=SN, logger=logger)
teach_arm.robot.resting()


In [ ]:
teach_arm.robot.set_enable(True)
teach_arm.robot.move_j(       -16.527421951293945,
      55.114776611328125,
      26.215545654296875,
      0.0007031249697320163,
      69.51187896728516,
      3.933046817779541)

In [ ]:
teach_arm.robot.set_enable(True)
teach_arm.robot.move_j(       -16.49370574951172,
      49.11359405517578,
      26.32851791381836,
      3.4315928587602684e-06,
      71.90644836425781,
      3.8402748107910156)